In [1]:
import os
import openai
import numpy as np
import pandas as pd
import ast

# Import and pass api key
with open(r'C:\Users\mhuh22\Desktop\openai_api.txt', 'r') as file:
    api_key = file.readline().strip()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

### Hallucinations
- when the model has enough information to phrase a response confidently, but incorrectly

In [3]:
prompt = "What does the start-up company Pentera do and who invested in it?"

response = openai.Completion.create(
    prompt = prompt,
    temperature = 0,
    max_tokens = 512,
    model = 'text-davinci-003'
)

print(response['choices'][0]['text'])



Pentera is a start-up company that provides software solutions to help organizations manage their employee benefits programs. The company has raised $3.5 million in seed funding from investors including Y Combinator, SV Angel, and Social Leverage.


Preventing Hallucination - no answers provided

Issues
* May not present correct information either

In [6]:
prompt = '''
Only answer the question below if you have 100% certainty of the facts.
Q: What does the start-up company Pentera do and who invested in it?
A: 
'''

In [7]:
response = openai.Completion.create(
    prompt = prompt,
    temperature = 0,
    max_tokens = 512,
    model = 'text-davinci-003'
)
print(response['choices'][0]['text'])

I do not have 100% certainty of the facts, so I cannot answer this question.


In [8]:
pwd

'C:\\Users\\mhuh22\\Documents\\Python\\Udemy'

In [15]:
df = pd.read_csv(r'UPDATED-MARCH-2023-Notebooks-and-Files\UPDATED-MARCH-2023-Notebooks-and-Files\09-Text-Embedding\unicorns.csv')

In [17]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN


In [21]:
df['Investors'][0]

'["Next Play Ventures","Zeal Capital Partners","SoftBank Group"]'

In [22]:
for investor in ast.literal_eval(df['Investors'][0]):
    print(investor)

Next Play Ventures
Zeal Capital Partners
SoftBank Group


In [68]:
def summary(company,crunchbase_url,city,country,industry,investor_list):
    investors = 'The investors in the company are'
     
    for investor in ast.literal_eval(investor_list):
        investors += f" {investor}, "

    text = f"{company} has headquarters in {city} in {country} and is in the field of {industry}. {investors}. You can find more information at {crunchbase_url}"

    return text 

In [69]:
df['summary'] = df.apply(lambda df: summary(df['Company'],df['Crunchbase Url'],df['City'],df['Country'],df['Industry'],df['Investors']),axis=1)

In [70]:
df['summary'][0]

'Esusu has headquarters in New York in United States and is in the field of Fintech. The investors in the company are Next Play Ventures,  Zeal Capital Partners,  SoftBank Group, . You can find more information at https://www.cbinsights.com/company/esusu'

In [37]:
import tiktoken

In [40]:
def num_token_from_string(string,encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [48]:
# 1000 tokens ~ 750 words
num_tokens = num_token_from_string(df['summary'][0], 'cl100k_base')

# Len of string 
num_characters = len(df['summary'][0])
num_words = len(df['summary'][0].split())

print(f"Characters: {num_characters} \nWords: {num_words} \nTokens: {num_tokens}")

Characters: 253 
Words: 38 
Tokens: 58


In [51]:
df['token_count'] = df['summary'].apply(lambda text: num_token_from_string(text,'cl100k_base'))

In [52]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,58
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,60
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,57
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,62
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,58


In [59]:
# Total tokens = 70,421
token_count = df['token_count'].sum()

# $cost = $0.0004 per 1000 tokens USD or 0.04c
cost = round(token_count * 0.0004 / 1000,4)

print(f"{token_count} tokens cost ${cost} USD")

70421 tokens cost $0.0282 USD


In [103]:
def get_embedding(text):
  # Note how this function assumes you already set your Open AI key!
    result = openai.Embedding.create(
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]


In [80]:
# # Apply embedding for each company - takes a while, and costs about 0.0282
# df['embedding'] = df['summary'].apply(get_embedding)

In [94]:
df = pd.read_csv(r'UPDATED-MARCH-2023-Notebooks-and-Files\UPDATED-MARCH-2023-Notebooks-and-Files\09-Text-Embedding\unicorns_with_embeddings.csv')
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,58,"[0.01195491198450327, -0.017717931419610977, -..."
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,60,"[0.009171437472105026, 0.01314949057996273, -0..."
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,57,"[0.002730059437453747, -0.03737899661064148, 0..."
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,62,"[-0.0024771858006715775, -0.024587858468294144..."
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,58,"[0.011331121437251568, -0.011193273589015007, ..."


In [104]:
# vector = get_embedding(df['summary'][0])
prompt = "What does the company Pentera do and who invested in it?"
prompt_embedding = get_embedding(prompt)

In [105]:
# There are other services/programs for larger amount of vectors
# Take a look at vector search engines like Pinecone or Weaviate
def vector_similarity(vec1,vec2):
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(vec1), np.array(vec2))

In [107]:
# df["prompt_similarity"] = df['embedding'].apply(lambda vector: vector_similarity(vector, prompt_embedding))